# Package

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython import display

# Mesh

In [ ]:
def mesh(h, lamb):
    k = lamb * h
    x = np.arange(-3, 3, h)
    x = np.append(x, np.array([3]))
    
    initial = initial_cond(x)
    
        
    t = np.arange(0, 2, k)
    t = np.append(t, np.array([2]))
    
    
    xx, tt = np.meshgrid(x, t)
    exact_mesh = xx - t[:, np.newaxis]
    
    u = np.zeros_like(xx)
    u[0, :] = initial
    return x, (u, exact_mesh)

# Initial, exact solution

In [ ]:
def initial_cond(x):
    y = np.cos(np.pi * x) ** 2
    y[np.abs(x)>0.5] = 0
    return y

In [ ]:
def exact_sol(mesh):
    y = np.cos(np.pi*mesh)**2
    for i in range(0, mesh.shape[0]):
        y[i,:][np.abs(mesh[i,:]) > 1/2] = 0
    return y

# Plotting

In [ ]:
def plotting_anim(x, ux, title='', skip_rate=1):
    fig = plt.figure()
    ax = fig.add_axes([0.1,0.1,0.8,0.8])
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$u(t, x)$')
    ax.grid(True)
    ax.set_title(title)
    
    ux = np.asarray(ux)
    ux = ux[:, ::skip_rate, :]
    
    lines = []
    for u, fmt, l in zip(ux, ['--b', '-r'], ['NSol', 'Exact']):
        line, = ax.plot([], [], fmt, label=l, alpha=0.5)
        lines.append(line)
        ax.set_xlim(-3, 3)
        ax.set_ylim(-0.5, 3.0)
    fig.legend()

    def animate(frame_num):
        ax.set_title(f'{title} FRAME={frame_num * skip_rate:04d}')
        for line, u in zip(lines, ux):
            y = u[frame_num, :]
            line.set_data((x, y))
        return lines

    anim = FuncAnimation(fig, animate, frames=ux[0].shape[0], interval=100)
    video = anim.to_jshtml()
    html = display.HTML(video)
    display.display(html)
    plt.close()

# Numerical Scheme

## Forwad-time backward-space

$$ \frac{v_{m}^{n+1} - v_{m}^{n}}{k} + a\frac{v_{m}^{n} - v_{m-1}^{n}}{h} = 0$$
&harr; $v_{m}^{n+1} = (1 - \lambda)v_{m}^{n} + \lambda v_{m-1}^{n} $

In [ ]:
def ftbs(h, lamb):
    x ,(u, exact) = mesh(h, lamb)
    
    exactsol = exact_sol(exact)
    for i in range(0, u.shape[0]-1):
        u[i+1, 1:] = (1-lamb)*u[i, 1:] + lamb*u[i, :-1]
        
    return x, (u, exactsol)

## Forwad-time Central-space

$$ \frac{v_{m}^{n+1} - v_{m}^{n}}{k} + a\frac{v_{m+1}^{n} - v_{m-1}^{n}}{2h} $$
$$v_{m}^{n+1} = v_{m}^{n} + \frac{\lambda}{2}(v_{m-1}^{n} - v_{m+1}^{n})$$

In [ ]:
def ftcs(h, lamb):
    x, (u, exact) = mesh(h, lamb)

    exactsol = exact_sol(exact)
    for i in range(0, u.shape[0]-1):
        u[i+1, 1:-1] = u[i, 1:-1] +  (u[i, 0: -2] - u[i, 2:]) * (lamb) * 0.5

    return x, (u, exactsol)

## Lax-Friedrichs

$$ \frac{v_{m}^{n+1} - \frac{1}{2}\left(v_{m+1}^{n}+v_{m-1}^{n}\right)}{k} + a\frac{v_{m+1}^{n} - v_{m-1}^{n}}{2h} $$
$$ v_{m}^{n+1} = \frac{1}{2}(1-\lambda)v_{m+1}^{n} + \frac{1}{2}(1+\lambda)v_{m-1}^{n}$$

In [ ]:
def lax_fried(h, lamb):
    x, (u, exact) = mesh(h, lamb)
    exactsol = exact_sol(exact)
    for i in range(0,u.shape[0]-1):
        u[i+1, 1:-1] = 1/2 * (1-lamb)*u[i, 2:] + (1/2)*(1+lamb)*u[i, 0:-2]
    return x, (u, exactsol)

## Leapfrog
$$ \frac{v_{m}^{n+1} - v_{m}^{n-1}}{2k} + a\frac{v_{m+1}^{n} - v_{m-1}^{n}}{2h} $$
$$ v_{m}^{n+1} = v_{m}^{n-1} -\lambda( v_{m+1}^{n} - v_{m-1}^{n})$$
Only `2-step` in this problem set. So I used first step, Lax - Friedrichs.

In [ ]:
def leapfrog(h, lamb):
    x, (u, exact) = mesh(h, lamb)
    exactsol = exact_sol(exact)
    u[1, 1:-1] = 1/2 * (1-lamb)*u[0, 2:] + (1/2)*(1+lamb)*u[0, 0:-2]
    for i in range(1, u.shape[0]-1):
        u[i+1, 1:-1] = u[i-1,  1:-1] - lamb*(u[i, 2:] - u[i, :-2])
    return x, (u, exactsol)

# Result

### Forward-time backward-space

In [ ]:
x, ux = ftbs(0.1, 0.8)
plotting_anim(x, ux, title='Forward-time backward-space(h=0.1)', skip_rate=1)

In [ ]:
x, ux = ftbs(0.05, 0.8)
plotting_anim(x, ux, title='Forward-time backward-space(h=0.05)', skip_rate=1)

In [ ]:
x, ux = ftbs(0.025, 0.8)
plotting_anim(x, ux, title='Forward-time backward-space(h=0.025)', skip_rate=1)

### Forward-time central-space

In [ ]:
x, ux = ftcs(0.1, 0.8)
plotting_anim(x, ux, title='Forward-time central-space(h=0.1)', skip_rate=1)

In [ ]:
x, ux = ftcs(0.05, 0.8)
plotting_anim(x, ux, title='Forward-time central-space(h=0.05)', skip_rate=1)

In [ ]:
x, ux = ftcs(0.025, 0.8)
plotting_anim(x, ux, title='Forward-time central-space(h=0.025)', skip_rate=1)

## Lax-Friedrichs

### $\lambda = 0.8$

In [ ]:
x, ux = lax_fried(0.1, 0.8)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.1)', skip_rate=1)

In [ ]:
x, ux = lax_fried(0.05, 0.8)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.05)', skip_rate=1)

In [ ]:
x, ux = lax_fried(0.025, 1)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.025)', skip_rate=1)

### $\lambda = 1.6 $

In [ ]:
x, ux = lax_fried(0.1, 1.6)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.1)', skip_rate=1)

In [ ]:
x, ux = lax_fried(0.05, 1.6)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.05)', skip_rate=1)

In [ ]:
x, ux = lax_fried(0.025, 1.6)
plotting_anim(x, ux, title='Lax-Friedrichs(h=0.025)', skip_rate=1)

## Leapfrog

In [ ]:
x, ux = leapfrog(0.1, 0.8)
plotting_anim(x, ux, title='leapfrog(h=0.1)', skip_rate=1)

In [ ]:
x, ux = leapfrog(0.05, 0.8)
plotting_anim(x, ux, title='leapfrog(h=0.05)', skip_rate=1)

In [ ]:
x, ux = leapfrog(0.025, 0.8)
plotting_anim(x, ux, title='leapfrog(h=0.025)', skip_rate=1)